<a href="https://colab.research.google.com/github/9StoneData/FUZZY_WUZZY/blob/main/DISP_2_MAN_MAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-Levenshtein
!pip install fuzzywuzzy
!pip install num2words


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
global CORES
import multiprocessing
CORES=multiprocessing.cpu_count()
import re

from num2words import num2words
import pandas as pd
pd.set_option('display.max_rows', 500)
import concurrent.futures
import datetime as dt
import time
import numpy as np
import sys
import gc
import random
import os
import pandas as pd
from os import listdir
from os.path import isfile, join

from fuzzywuzzy import process,fuzz

# Problem Statement:

Each day, the Leafly Project pulls over 90,000 product listings.  The products have inconsistent formats: "a b c vs ab c vs cb a" , etc.  The data is only valuable if it can be analyzed.  Also, it is my intention to monetize this data and build out a dashboard SaSS.  I need to store it in database, and I want to have clean tables.  

Solution:

I have been refining the "Humpty Dumpty" or for the sake of this writing, "Humpty Data" approach.  Looking back at the process, this makes me laugh.  As in the nursery ryhme, Humpty Dumpty fell off the wall.  However, with my relentless know enough to be dangerous approach, I pushed the data off the wall first.  

Here is how...

At first, I saw that Leafly also listed products separately from the dispensaries.  So, lets take a few steps back to give ourselves some room for a running start.  I decided to build a scraper to gather those listings thinking I could just match them up with some sort of python magic.

As I am about to take off with a full head of steam, I am stopped by a wizard from Levenshtein-ville.  Would you like to like to use this magic I have to help with your quest?  A yeah!!!  Here I go with a full head of steam and a new python weapon, the fuzzywuzzy library, and I am going to conquer this problem!!!

And I hit "Humpty Data" with everyting my stubborn mind had.  What is I compare this name to that one.  Now I need to asynchrnously process this...  Look its working... Its alive.  And now, its gone.  Play montouge....  I am back, lets do this again.  Now lets try weighting the searchers, load the names with aditional information... No, lets try building a parsrer with a library I never even used before.  Two days later, I discoverd gold in them there hills.  Cascadeing parser.  Cleans the names and captures key data.  Ok, I am about to get Mr. "Humpty Data" off the wall.  

Refector some code, fight off a bug or 100, and BOOM!  I have my "clean list", Humpty Data has gone down.... so I thoght.  I over shot the mark and just created more work.   I reduced the names down to similar names, but now I had grouped them too well.  Turns out you need the weights and ratios in the name... Who knew.  Here I am looking over my Data all over the creek bed as it fell of the bridge, thinkning to my self what had I done.

This time all the King's horses and all of the King's men will put Humpty Data back together again...



# STEP 1: Clean formatting

1. Remove manufacturer names from the products
2. Parse common "TAGS" out of the product name

Call it The Humpty Dumpty algorithim.  But this time all of the King's horses and all of the King's men could put Humpty Dumpty back together again!  

In [ ]:
#Read in man adjusted product list
#SDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_renamed_mans.csv",dtype={'Manufacturer':str,"Product":str})
SDF =pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_.csv")
#clean out nan's
SDF.loc[SDF['Product_size'].isna(),'Product_size']=""
SDF.loc[SDF['Product'].isna(),'Product_size']=""
SDF.loc[SDF['Manufacturer'].isna(),'Product_size']=""
SDF = SDF.dropna(subset=SDF.select_dtypes(np.floating).columns, how='all')
SDF = SDF.dropna(subset=['Manufacturer','Product'])
SDF=SDF.reset_index()
SDF=SDF.drop(['index'],axis=1)
#build search term
SDF=SDF.copy()
SDF['query_dirty']=SDF['Product']+" "+SDF['Product_size']
SDF['query_dirty']=SDF['query_dirty'].str.replace('|',' ',regex=True)
SDF['query_dirty']=SDF['query_dirty'].str.replace('(',' ',regex=True)
SDF['query_dirty']=SDF['query_dirty'].str.replace(')',' ',regex=True)
SDF['query_dirty']=SDF['query_dirty'].str.replace('-',' ',regex=True)
SDF['query']=SDF['query_dirty'].str.lower()

SDF['Manufacturer_chk']=SDF['Manufacturer'].str.replace('*','',regex=True)
SDF['Manufacturer_chk']=SDF['Manufacturer'].str.replace('-','',regex=True)
SDF["TAG"]=""
print(len(set(SDF['query'])))
products=list(SDF['query'])
corrected=0
updated=0
adjusted=0
t1=t2=t3=t4=t5=t6=0
sizes=0
total_changed=0
for i,product in enumerate(products):
  test=product
  product=product.lower().replace('*flash sale*','')
  maker=SDF.at[i,'Manufacturer_chk'].lower().replace(".","").replace('  ',' ').replace(' AZ','').strip('*')
  TEST=True

  #Cascade :
  original=product
  #TEST1
  MATCHED=False
  if len(product)>len(maker):
    test_maker=maker.split()
    test_product=product.split()
    matches=[]
    for m,mk in enumerate(test_maker):
      for pr,prd in enumerate(test_product):
        if mk=="grön" and prd=="gron":
          matches.append("gron")
          MATCHED=True
        if prd=="alien" and pr<len(test_product) and test_product[pr+1]=="labs":
          matches.append("alien labs")
          MATCHED=True
        elif mk.replace("'","").replace(".","")==prd.strip(":").replace("'","").replace(".","").replace('!',''):
          matches.append(prd)
          MATCHED=True
          break
    for mt in matches:
      product=product.replace(str(mt),'',1)
    product=product.lstrip(" ").lstrip(":").replace("  "," ").rstrip(" ")
    if MATCHED:
      t1+=1
      total_changed+=1
  # if MATCHED and i%5==0:
  #   print(f"[p={i:5} {t1:5} {t1/(len(products))*100:2.2f}% {total_changed/(len(products))*100:2.2f}% ]{maker:20} ===> {product:60} {original:60}")

  
  units= {
    "Fractions"    :r"(\s{1}\d{0,4}\s{0,1}\d{1,4}\s{0,1}\/\s{0,1}\d{1,4}\s{0,1}(g|oz|mg))",
    "Ratios"       :r"((\d{1,5}\s{0,1}((thc:cbd|cbd:thc|thc:cbd:cbn|mg|g\s|thc|cbd|cbg|cbn){0,2}\s{0,1}){1,2})((\/|:)\s{0,1}\d{1,5}\s{0,1}((thc\s{0,1}:\s{0,1}cbd|cbd:thc|mg|g\s|thc|cbd|cbg|cbn){0,2}\s{0,1}){1,2}){1,4})",
    "Pack_size"    :r"(((\d{1,4}\s{0,1}(pk|pack))|(\s\d{0,3}\s{0,1}\.{0,1}\d{0,2}(g\s|mg\s|oz\s))\s{0,1}x\s{0,1}\d{1,4})|((\s\d{0,3}\s{0,1}\.{0,1}\d{0,2}x\.{0,1}\s{0,1}\d{1,4}(g|mg|oz|ml))\s{0,1}))",
    'Weight'       :r"(\d{0,5}\s{0,1}[/.]{0,1}\s{0,1}\d{1,5}(mg|ml|g|oz)\s{0,1})",
    'Category_TAG' :r"((flower|cartridge|hybrid|\sh\s|\si\s|\ss\s|\si\/h|\sd\/icartridges|preroll|preerolls|pods|pod|pre pack|pre roll|pre rolls|resin|rosin|tablets|hybrid|sativa|indica){1,10})"

  }
  #I know: ugly but do not want to spend an hour redoing regex
  product=product+" " 
  for k,v in units.items():
    result = re.findall(v, product)  
    original = product
    if result:
      for res in result:
        product=product.replace(res[0],' ',1)
        SDF.at[i,"TAG"]+=f"+{k}<{res[0].replace(' ','').replace('pack','pk')}>"
        SDF.at[i,'BASE_NAME']=product.replace("[","").replace("]","").replace(":","").replace("  "," ").lstrip(" ").rstrip(" ")
        if k!='Fractions':
          SDF.at[i,k]=res[0].replace(' ','').replace('pack','pk')
        else:
          SDF.at[i,"Weight"]=res[0].replace(' ','').replace('pack','pk')
        if i%20==0:
          print(f"{i:5} {k:20} {res[0]:20} >><<< {original:60}<<======>>{SDF.at[i,'BASE_NAME']}")

PRODUCTS=set(SDF['query'])
print(len(PRODUCTS))
SDF.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_renamed_products_NO_MAN.csv",index=False)

# Problem Statement

When collecting data from Leafly.com, products are listed in two places.  First, they are listed as offferings from dispensaries.  Second, they are listed under the product section categorized by type and sub category.  

Dispensaries advertise the same product from the same manufacturer with different variations of the manufacturere and product name, and or they add additional information to the names, i.e. product size, flavor, price, or promotions.  

An additional issue is that under the product section, not all of the manufacturers and their respective product catelogs are listed.

If one is to analize products from Leafly, one must homogenize thier names.  In other spaces such as the Automotive Retail Space or Grocery Space, products have either a VIN or a UPC allowing one to analyize or group identicle products.  Leafly does not have this.  

The output from the despencary product level is non homeginzed preventing accurate analysis of the products.  

#Solution

Utlizing the fuzzy library with the Levenshtein algorithim  and various data cleansing techniques, develop a homogenized list of products and manufacturer names. 

The list of products at dispensaries is gathered at the State Market Level.  For Arizone, there are over 90k products advertised.  From the product page scrape, there are over 60k different products.  Multi thread processing will be required to effeciently process and map the Manufacturer and Product names so one can have confidence in their analyis.

## Part 1.1: Read in Products from Leafly.com product scrape:  

These are catorgized by product category and will serve as the main name for Manufacturer and Product.  Check to see if there are variations in the Manufacturer name to ensure homogenized and clean data.


In [ ]:
MAN_PROD=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_PROD_PAGE_DETAILS_.csv")
MAN_PROD.info()
MAN_PROD.loc[MAN_PROD['Manufacturer'].isna(),'Manufacturer']="DO NOT KNOW FOR SURE"

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60680 entries, 0 to 60679
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   category_url      60680 non-null  object 
 1   THREAD            60680 non-null  int64  
 2   total_pages       60680 non-null  int64  
 3   sub-category      60680 non-null  object 
 4   category          60680 non-null  object 
 5   srp_products      60680 non-null  object 
 6   page              60680 non-null  float64
 7   pdv               60680 non-null  object 
 8   True_Product      60680 non-null  object 
 9   Manufacturer      60680 non-null  object 
 10  rating            60680 non-null  float64
 11  count_of_ratings  60680 non-null  float64
 12  Collected_at      60680 non-null  object 
 13  Page_Load_Tries   60680 non-null  float64
dtypes: float64(4), int64(2), object(8)
memory usage: 6.5+ MB


## Part 1.2: Keep columns of interest:  

Columns of interest:

*   pdv          : product detail page url
*   True_Product : Name of product
*   Manufacturer : Name of Manufacturer
*   category     : Main product category (Cannabis, Edibles,Vapes, etc)
*   sub-category : preroll, sublingual, cookies, etc

In [ ]:
MAN_PROD=MAN_PROD[['Manufacturer','True_Product','category','sub-category','pdv']]
MAN_PROD.columns

Index(['Manufacturer', 'True_Product', 'category', 'sub-category', 'pdv'], dtype='object')

## Part 2.1: Check for duplicate manufacturers:  

Make sure the same manufacturer is not listed multiple times with slight variaotiosn to their name, i.e. LOVELY and LOVELI and LOVELY inc.


In [ ]:

#list of manufacturers
mans=list(set(MAN_PROD['Manufacturer']))

#DF lookup table for competing names
VC=pd.DataFrame(MAN_PROD['Manufacturer'].value_counts())

#depending on cpu use, Process pool or Thread pool will be used for asynchronous processing
#keep in mind that a=b in one thread, b=a in another or in a different search.  resolve this
# by picking the one with more products as the master name.  will require manual review of changes
# but, can autmate the process with screen input once out of the step


#Prepair data for asynchrounous processing
THREADS=CORES

query  = mans.copy()
search = mans.copy()

QUERY  = pd.DataFrame(query ,columns=['Manufacturer'])
SEARCH = pd.DataFrame(search,columns=['Manufacturer'])

#configure 
QUERY['THREAD']=QUERY.index%THREADS
MASTER_LIST=[]
for i in range(THREADS):
  temp  = []
  tdf   = QUERY[QUERY['THREAD']==i]
  #send copies not pointers to original
  tdf   = tdf.copy()
  #easier to reference rows
  tdf.reset_index(inplace=True)
  tdf.drop(['index'],axis=1,inplace=True)
  tdf['query_column'] = 'Manufacturer'
  tdf['query_name']="product_manufacturer_vs_product_manufacturer_.csv"
  tdf['directory'] = "/content/drive/MyDrive/LEAFLY_Project/Data/THREAD_RESULTS/MAPPING"
  tdf['Multiple_Possible_Names']=False
  tdf['MASTER_MAN_NAME']=tdf['Manufacturer']
  tdf['algo_step']=1
  SEARCH = SEARCH.copy()
  SEARCH['search_column']='Manufacturer'
  temp.append(tdf)
  temp.append(SEARCH)
  VC=VC.copy()
  temp.append(VC)
  MASTER_LIST.append(temp)

def process_for_match(LIST_o_DF):

  try:

    QUERY_DF          = LIST_o_DF[0]
    q1_LIST_column    = QUERY_DF.at[0,'query_column']
    q1_LIST           = list(QUERY_DF[q1_LIST_column]) 

    SEARCH_DF         = LIST_o_DF[1]
    s1_LIST_coliumn   = SEARCH.at[0,'search_column']
    s1_LIST           = list(set(SEARCH_DF[s1_LIST_coliumn]))

    VC                = LIST_o_DF[2]
  except:
    print("error initializing dfs")

  try:
    if QUERY_DF.at[0,'algo_step']==1:
      for q,query in enumerate(q1_LIST):
        
        try:
          ops=process.extract(query,s1_LIST)
        except:
          print("oops error",query)

        master=query
     
        if len(ops)>0:
          
          for o,op in enumerate(ops):
            EXACT=False
            #assuming forst one is the same, but can check
            if query==op[0]:
              EXACT=True
        
            #ran various thresholds.  92 + is least number of false positives
            if not EXACT and  op[1]>=92:
              #Decide on master name by which one hsows up more 
              man1=query
              man2=op[0]
    
              if VC.at[man2,'Manufacturer'] > VC.at[man1,'Manufacturer']:
                master=man2
              print(f"{q:3} {query:40} =?= {op[0]:40} {VC.at[man1,'Manufacturer']:5} {VC.at[man2,'Manufacturer']:5} {master}")
              QUERY_DF.at[q,'Multiple_Possible_Names']=True
              QUERY_DF.at[q,f"_{o}_possible_name"]=op[0]
              QUERY_DF.at[q,"Number_of_Possible_Names"]=o
              QUERY_DF.at[q,'MASTER_MAN_NAME']=master
              
              
        
        # if q%10==0:
        #   print(ops)
      #by design, as the DF's are returned, the master df is rebuilt
  except:
    print("Error getting ops")


  return(QUERY_DF,SEARCH_DF)

def process_for_match_THREADED(MASTER_LIST):
  threads=len(MASTER_LIST)

  FDF=pd.DataFrame()
  FMMDF=pd.DataFrame()

  with concurrent.futures.ProcessPoolExecutor(max_workers=threads) as executor:
    try:
        for r,results in enumerate(executor.map(process_for_match,MASTER_LIST)):
          FDF=FDF.append(results[0],ignore_index=True)
        
        path=FDF.at[0,'directory']
        file_name=FDF.at[0,'query_name']
        file_date_stamp=dt.datetime.now()
        file_date_stamp=file_date_stamp.strftime("%m_%d_%Y_%H_%M_")
        FDF.to_csv(join(path,f"_{file_name}"),index=False)
        FDF.to_csv(join(path,f"_{file_date_stamp}_{file_name}"),index=False)
        print(FDF.info())

    except:
        print("OOPS")

  return FDF


In [ ]:
FDF=process_for_match_THREADED(MASTER_LIST)

  3 flowerz                                  =?= Flower                                       5    15 Flower
 25 Ottos OG Farm                            =?= Otto's OG Farm                               1     3 Otto's OG Farm
 70 Cultivar Collection                      =?= Cultivators Collection                       7     5 Cultivar Collection
113 Otto's OG Farm                           =?= Ottos OG Farm                                3     1 Otto's OG Farm
188 Capital Cannabis                         =?= Capital City Cannabis                       27     1 Capital Cannabis
188 Capital Cannabis                         =?= Cannabis Capital                            27     2 Capital Cannabis
279 Dhgate                                   =?= DH Gate                                      1     1 Dhgate
291 Harvest of Tucson                        =?= Harvest of Tuscon                            5     2 Harvest of Tucson
302 Peak Supply Co.                          =?= Peak Supply        

## Part 2.2: Overwrite manufacturer names with Master Name, and remove uncessary columns

Resulting columnns:
* 'Manufacturer'             : original name
* *'THREAD'*
* *'query_column'* 
* *'query_name'* 
* *'directory'*
* *'Multiple_Possible_Names' * 
* 'MASTER_MAN_NAME'          : Name to use for all matchtes
* '_1_possible_name' 
* 'Number_of_Possible_Names' 
* '_0_possible_name' 
* '_2_possible_name'


In [ ]:
#Original Product Scrped Table = MAN_PROD
#reloading for the sake of not having to scroll back to top to run again
MAN_PROD=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_PROD_PAGE_DETAILS_.csv")

FDFa = FDF[['Manufacturer','MASTER_MAN_NAME','Number_of_Possible_Names']]
print(f" Number of unique Manufactureres___________: {FDFa.shape[0]:4}")
#FDFa=FDFa.drop_duplicates(subset=['MASTER_MAN_NAME'],keep='first')

temp = FDFa[FDFa['Manufacturer']!=FDFa['MASTER_MAN_NAME']]
print(f" Manufacturers to correct__________________: {temp.shape[0]:4}")


tofix=0
for man in list(temp['Manufacturer']):
  test=MAN_PROD[MAN_PROD['Manufacturer']==man]
  tofix+=len(test)

corrections=0
for man in list(temp['Manufacturer']):
  tdf=(MAN_PROD.loc[MAN_PROD['Manufacturer']==man,'Manufacturer'])
  corrections+=len(tdf)
  MAN_PROD.loc[MAN_PROD['Manufacturer']==man,'Manufacturer']=temp.loc[temp['Manufacturer']==man,'MASTER_MAN_NAME']
  #tdf=(MAN_PROD.loc[MAN_PROD['Manufacturer']==man,'Manufacturer'])
  # print(f"{man} :: {len(tdf)} :: {temp.loc[temp['Manufacturer']==man,'MASTER_MAN_NAME']} ::::::::")
  #print(f"{temp.loc[temp['Manufacturer']==man,'MASTER_MAN_NAME']} ::::::::")

errors=0
for man in list(temp['Manufacturer']):
  test=MAN_PROD[MAN_PROD['Manufacturer']==man]
  if len(test)>0:
    print(man)
  errors+=len(test)
print(f" Products with Manufacturer name adjusted__: {corrections:4} of {tofix} with {errors} missed")

MAN_PROD.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/MASTER_PRODUCT_LSIT_renamed_mans.csv",index=False)
FDFa.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/MASTER_PRODUCT_LSIT_keys.csv",index=False)

 Number of unique Manufactureres___________: 3077
 Manufacturers to correct__________________:   27
 Products with Manufacturer name adjusted__:  141 of 141 with 0 missed


## Part 3.1: Start Mapping Dispensary products manufactureres to Master List
##Chose not to pair up with product page names... just going to build database from cleaned product names from Dispensary page.  


Scrapes generate a csv of Dispensary Catelog of products.  Most have a manufacturer listed.  

Need to build a table to map them to Master Manufacturer List.  Very similar process as step 2, just need to reconfigure MASTER_LIST

Scraper runs 1 to many State Markets a day based on time constraints and inputs.  Each state will have to be initially processed to build mapping table, and updated from time to time based ona schedule

first step is to homeginze the disp manf names to itself 

In [ ]:
SDF =pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_.csv")
MAN_PROD=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/MASTER_PRODUCT_LSIT_renamed_mans.csv")

print(SDF.shape)
SDF.loc[SDF['Manufacturer'].isna(),'Manufacturer']="TBD9999999999999999999999999999888888888888888888888888888888"
SDF.loc[SDF['Manufacturer']=="-",'Manufacturer']="TBD99999999999999999999999999999999999999999999999999"
#list of manufacturers
disp_mans=list(set(SDF['Manufacturer']))
mans=list(set(SDF['Manufacturer']))


#DF lookup table for competing names
VC=pd.DataFrame(SDF['Manufacturer'].value_counts())

#depending on cpu use, Process pool or Thread pool will be used for asynchronous processing
#keep in mind that a=b in one thread, b=a in another or in a different search.  resolve this
# by picking the one with more products as the master name.  will require manual review of changes
# but, can autmate the process with screen input once out of the step


#Prepair data for asynchrounous processing
THREADS=CORES

query  = disp_mans.copy()
search = mans.copy()

QUERY  = pd.DataFrame(query ,columns=['Manufacturer'])
SEARCH = pd.DataFrame(search,columns=['Manufacturer'])

#configure 
QUERY['THREAD']=QUERY.index%THREADS
MASTER_LIST=[]
for i in range(THREADS):
  temp  = []
  tdf   = QUERY[QUERY['THREAD']==i]
  #send copies not pointers to original
  tdf   = tdf.copy()
  #easier to reference rows
  tdf.reset_index(inplace=True)
  tdf.drop(['index'],axis=1,inplace=True)
  tdf['query_column'] = 'Manufacturer'
  tdf['query_name']="product_manufacturer_vs_vendor_manufacturer_.csv"
  tdf['directory'] = "/content/drive/MyDrive/LEAFLY_Project/Data/THREAD_RESULTS/MAPPING"
  tdf['Multiple_Possible_Names']=False
  tdf['MASTER_MAN_NAME']=tdf['Manufacturer']
  tdf['algo_step']=1
  SEARCH = SEARCH.copy()
  SEARCH['search_column']='Manufacturer'
  temp.append(tdf)
 
  temp.append(SEARCH)
  VC=VC.copy()
  temp.append(VC)
  MASTER_LIST.append(temp)


FDF=process_for_match_THREADED(MASTER_LIST)



(93715, 48)
  0 AlienLabs                                =?= Alien Labs                                  32   464 Alien Labs
  0 WWW.SHIZNIC.COM                          =?= Shiznic.com                                  1     5 Shiznic.com
  0 Evolution Farms                          =?= EVOLUTION FARMS                              1     2 EVOLUTION FARMS
  1 AtoZ Botanical                           =?= AtoZ Botanicals                              1     2 AtoZ Botanicals
  2 STIIIZY                                  =?= Stiiizy                                   4977     2 STIIIZY
  1 Bee Line Hemp Wick                       =?= Beeline Hemp Wick                            1     3 Beeline Hemp Wick
  2 SKUNK BRAND                              =?= SkunkBrand                                   2     5 SkunkBrand
  3 OGeez!                                   =?= OGeez                                        2     9 OGeez
  3 OGeez!                                   =?= Ogeez                    

# Step 3.2: Now Overwrite scraped names in Dispensary Product results

THIS CELL SHOULD BE a function to call for both of the similar steps...

In [ ]:
#Original Product Scrped Table = MAN_PROD
#reloading for the sake of not having to scroll back to top to run again
#MAN_PROD=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_PROD_PAGE_DETAILS_.csv")
SDF =pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_.csv")
FDFa = FDF[['Manufacturer','MASTER_MAN_NAME','Number_of_Possible_Names']]
print(f" Number of unique Manufactureres___________: {FDFa.shape[0]:4}")
#FDFa=FDFa.drop_duplicates(subset=['MASTER_MAN_NAME'],keep='first')

temp = FDFa[FDFa['Manufacturer']!=FDFa['MASTER_MAN_NAME']]
print(f" Manufacturers to correct__________________: {temp.shape[0]:4}")


tofix=0
for man in list(temp['Manufacturer']):
  test=SDF[SDF['Manufacturer']==man]
  tofix+=len(test)

corrections=0
for man in list(temp['Manufacturer']):
  tdf=(SDF.loc[SDF['Manufacturer']==man,'Manufacturer'])
  corrections+=len(tdf)
  SDF.loc[SDF['Manufacturer']==man,'Manufacturer']=temp.loc[temp['Manufacturer']==man,'MASTER_MAN_NAME']
  #tdf=(MAN_PROD.loc[MAN_PROD['Manufacturer']==man,'Manufacturer'])
  # print(f"{man} :: {len(tdf)} :: {temp.loc[temp['Manufacturer']==man,'MASTER_MAN_NAME']} ::::::::")
  #print(f"{temp.loc[temp['Manufacturer']==man,'MASTER_MAN_NAME']} ::::::::")

errors=0
for man in list(temp['Manufacturer']):
  test=SDF[SDF['Manufacturer']==man]
  if len(test)>0:
    print(man)
  errors+=len(test)
print(f" Products with Manufacturer name adjusted__: {corrections:4} of {tofix} with {errors} missed")

SDF.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_renamed_mans.csv",index=False)
FDFa.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_keys.csv",index=False)

 Number of unique Manufactureres___________:  961
 Manufacturers to correct__________________:  146
 Products with Manufacturer name adjusted__: 1281 of 1281 with 0 missed


# Step 3.3: Clean Product names (Clear out MAN name in products)

GOT MESSY but "it works"

Purppse:  Remove manufacturer names from products so when looking them up in the products table there is a higher score and less likely to add to the table.

Product table does not include manufacturer name in the product

Adding regex expression to find UOM's and wieghts, thc,cdb ratios

possible change to algorithim order.  keep oringal list to clean outmanufacturer names so more match woth product name test


# STEP 3.4

Parse UOM out of product **descriptions**

Cleaned product name is in ['query']

In [ ]:
SDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_renamed_products_NO_MAN.csv")
products_o=len(set(SDF['Product']))
SDF=SDF.drop_duplicates(subset="Product_Url",keep='first')
SDF=SDF[~SDF['query'].isna()]
SDF = SDF.dropna(subset=['query'])
SDF['TAG']=""
SDF['BASE_NAME']=SDF['query']
SDF=SDF.reset_index()
SDF=SDF.drop(['index'],axis=1)
#Dictionary of dictionaries of various regex 
#-Pull out in cascading order: if a==>b and b!==>a do b before a

units= {
    "Fractions"    :r"(\s{1}\d{0,4}\s{0,1}\d{1,4}\s{0,1}\/\s{0,1}\d{1,4}\s{0,1}(g|oz|mg))",
    "Ratios"       :r"((\d{1,5}\s{0,1}((thc:cbd|cbd:thc|thc:cbd:cbn|mg|g\s|thc|cbd|cbg|cbn){0,2}\s{0,1}){1,2})((\/|:)\s{0,1}\d{1,5}\s{0,1}((thc\s{0,1}:\s{0,1}cbd|cbd:thc|mg|g\s|thc|cbd|cbg|cbn){0,2}\s{0,1}){1,2}){1,4})",
    "Pack_size"    :r"(((\d{1,4}\s{0,1}(pk|pack))|(\s\d{0,3}\s{0,1}\.{0,1}\d{0,2}(g\s|mg\s|oz\s))\s{0,1}x\s{0,1}\d{1,4})|((\s\d{0,3}\s{0,1}\.{0,1}\d{0,2}x\.{0,1}\s{0,1}\d{1,4}(g|mg|oz|ml))\s{0,1}))",
    'Weight'       :r"(\d{0,5}\s{0,1}[/.]{0,1}\s{0,1}\d{1,5}(mg|ml|g|oz)\s{0,1})",
    'Category_TAG' :r"((flower|cartridge|cartridges|preroll|preerolls|pods|pod|pre pack|pre roll|pre rolls|resin|rosin|tablets|hybrid|sativa|indica){1,10})"

}

for i in SDF.index:
  product=SDF.at[i,'query']
  
  for k,v in units.items():
    result = re.findall(v, product)  
    original = product
    if result:
      for res in result:
        product=product.replace(res[0],' ',1)
        SDF.at[i,"TAG"]+=f"+{k}<{res[0].replace(' ','').replace('pack','pk')}>"
        SDF.at[i,'BASE_NAME']=product.replace("[","").replace("]","").replace(":","").replace("  "," ").lstrip(" ").rstrip(" ")
        if k!='Fractions':
          SDF.at[i,k]=res[0].replace(' ','').replace('pack','pk')
        else:
          SDF.at[i,"Weight"]=res[0].replace(' ','').replace('pack','pk')
        if i%20==0:
          print(f"{i:5} {k:10} {res[0]:20} >><<< {original:60}<<======>>{SDF.at[i,'BASE_NAME']:60}")


SDF[['query','BASE_NAME','TAG','Weight','Ratios','Pack_size']].head(500)

products=len(set(SDF['BASE_NAME']))
print(products,products_o)
SDF.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_renamed_products_NO_MAN_Product_names_parsed.csv",index=False)

# STEP 3.5.1 Overwrite duplicated names 

(a=a. now a.='a' for a>a in occurance)

Challange:  500mg a and 1000mg a coming back to close to each other.  Tested overweighting 'a' with adding the uom's to the name.  This added more noise.  Going to test with regrex methods to see if 1000 and 500 show up.  if they do, going to reject the change.

First filter out the for sure not duplicats.  (There are alot of moving parts to decide on master name)



In [ ]:
def parse_products(LIST_o):
  """
  this could be built out to overwrite other columns to fill in blanks
  """
  #Find numbers
  find_numbers=r"((\.){0,1}\d{1,4}\.{0,1}\d{0,2})"


  To=time.time()
  DFO=LIST_o[0]
  ML=LIST_o[1]    #df of all product counts 
  s1_LIST=list(ML['index'])
  #print(DFO.shape)
  thread=DFO.at[0,'THREAD']
  RDF=pd.DataFrame()
  #Index(['index', 'query', 'THREAD']
  products=list(DFO['index'])


  for p,product in enumerate(products):
    
    tdf=DFO[DFO['index']==product]
    tdf=tdf.copy()
    tdf=tdf.reset_index()
    tdf=tdf.drop(['index'],axis=1)
    tdf.at[0,'product']=product
    tdf.at[0,'DUP_PRODUCT_NAME']="NO"
    try:
      
      try:
        ops=process.extract(product,s1_LIST)
      except:
        print("ops error",product)
      
      try:
        total=0
        MATCHED=False
        for o,op in enumerate(ops):
          if op[1]>=70:
            total+=1
            product1=product.replace(" ","")
            product2=op[0].replace(" ","")
            product1=product.replace(":","_to_")
            product2=op[0].replace(":","_to_")

            result = re.findall(find_numbers, product1)
            if result:
              for res in result:
                new=num2words(res[0])
                product1=product1.replace(res[0],' ',1)
                product1=product1+" "+new

            result = re.findall(find_numbers, product2)
            if result:
              for res in result:
                new=num2words(res[0])
                product2=product2.replace(res[0],' ',1)
                product2=product2+" "+new
      

            nop=process.extractOne(product1,[product2])
            token=fuzz.token_set_ratio(product1,product2)
            ratio=fuzz.ratio(product1,product2)
            part_rat=fuzz.partial_ratio(product1,product2)

            avg=(nop[1]+token+ratio+part_rat)/4
            v1=v2=0
            if total>1:
          
              tdf.at[0,'DUP_PRODUCT_NAME']="YES"
              v1=ML.loc[ML['index']==product,'BASE_NAME']
              v1=v1.iloc[0]
              v2=ML.loc[ML['index']==op[0],'BASE_NAME']
              v2=v2.iloc[0]
              v1o=v1
              v2o=v2
              if avg>86 :
                MATCHED=True
                if v2 >= v1:
                  v1=v2
                  NEW=op[0]
                else:
                  NEW=product
              
                # print(f"-------------------------------------------------------\
                # \nProduct[query ]: {product:65}  vs\
                # \nProduct[search]: {op[0]:65} possibles Scores:[{p:5} {op[1]:4} :: {nop[1]:4} {token:4} {ratio:4} {part_rat:4}:::AVERAGE: {avg:3.2f}] Counts:[{v1o:4} {v2o:4} {v1o/v2o:2.2f}]")
        
        
        if MATCHED:
          tdf.at[0,'MASTER_PRODUCT_NAME']=NEW   
        else:
          tdf.at[0,'MASTER_PRODUCT_NAME']=product
        RDF=RDF.append(tdf,ignore_index=True)
        
        if p %5==0:
          print(f"[{(p+1)/len(products)*100:3.2f}% Complete] In thread {thread:3} on {p:5} of {len(products):5} products: {product:50} with {total:3} ")
        # # try:
        #   test=RDF.loc[RDF['DUP_PRODUCT_NAME']=="YES",:]
        #   if p%50==0:
        #     Total_TIME=time.time()-To
        #     EST_Total_Time=Total_TIME/((p+1)/len(products))


        #     print(f"[{(p+1)/len(products)*100:6.2f}% Complete] [{len(test)/(p+1)*100:6.2f}% Duplicate] [{Total_TIME/3600:6.2f}Hrs {(EST_Total_Time-Total_TIME)/3600:4.2f} hrs left]==>> In thread {thread:3} on {p:5} of {len(products):5} product: {product:50} :: {total:3} ")
        # except:
        #   print("error getting test",product)
      
      except:
        print("error processing ops",product)
    
    except:
      print("oops error",product)
    # if len(tdf)<1:
    #   print("ohhhhhhhhhhhhhhhhhh")
    # print(tdf.shape)
    # DFO=pull_uoms(product,p,SDF)
    # else:
    #   query=tdf.at[0,'query']
    #   pull_uoms(query,p,tdf)
    #   RDF=RDF.append(tdf,ignore_index=True)

  return(RDF)


def MASTER_LIST_parse_products(THREADS):
  SDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_renamed_products_NO_MAN_Product_names_parsed.csv")
  print(SDF.columns)


  # SDF=SDF[['Dispensary','Product_size', 'Manufacturer', 'Category', 'Listed_content', 'THC%',
  #       'CBD%', 'UOM', 'Price', 'query','Manufacturer_chk','Product_Url','mg',
  #       'g', 'ratio', 'thc_cbd', 'pk_cont', 'pk', 'strain', 'oz']]

  
  print(SDF.shape)       
  print(SDF.columns)

  SDF=SDF.drop_duplicates(subset="Product_Url",keep='first')
  SDF=SDF[~SDF['BASE_NAME'].isna()]
  SDF = SDF.dropna(subset=['Manufacturer','Product_Url','BASE_NAME'])
  SDF=SDF.astype(str)
  """
  Thought about "overweighting the name to reduce 1000 to 500 matches, but caused more noise 
  """
  SDF['BASE_NAME']=SDF['BASE_NAME']+" :: "+SDF['Manufacturer']

  SDF=SDF.reset_index()
  SDF=SDF.drop(['index'],axis=1)

  print(SDF.shape)
  #SDF['query']=SDF['Manufacturer']+" "+SDF['query']
  VC=pd.DataFrame(SDF['BASE_NAME'].value_counts())
  VC=VC.reset_index()
  #make products the index
  VC['THREAD']=VC.index%THREADS
  MASTER_LIST=[]
  for i in range(THREADS):
    temp=[]
    tdf=VC[VC['THREAD']==i]
    print(tdf.shape,VC.shape,len(MASTER_LIST))
    tdf=tdf.copy()
    tdf=tdf.reset_index()
    #not index but level_0 since did not drop index last time
    tdf=tdf.drop(['level_0'],axis=1)
    tVC=VC.copy()
    temp.append(tdf)
    temp.append(tVC)
    MASTER_LIST.append(temp)
  print(tVC.columns) 
  return(MASTER_LIST)

def parse_products_THREADED(MASTER_LIST):
  threads=len(MASTER_LIST)
  FDF=pd.DataFrame()
  FMMDF=pd.DataFrame()
  with concurrent.futures.ProcessPoolExecutor(max_workers=threads) as executor:
    try:
        for r,results in enumerate(executor.map(parse_products,MASTER_LIST)):
          FDF=FDF.append(results,ignore_index=True)
        print("TESTING")
        # path=FDF.at[0,'directory']
        # file_name=FDF.at[0,'query_name']
        # file_date_stamp=dt.datetime.now()
        # file_date_stamp=file_date_stamp.strftime("%m_%d_%Y_%H_%M_")
        # FDF.to_csv(join(path,f"_{file_name}"),index=False)
        # FDF.to_csv(join(path,f"_{file_date_stamp}_{file_name}"),index=False)
        print(FDF.info())

    except:
        print("OOPS")
  return(FDF)


THREADS=CORES
MASTER_LIST=MASTER_LIST_parse_products(THREADS)
FDF=parse_products_THREADED(MASTER_LIST)
#queryy in the numbver of occurances
FDF.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_renamed_products_now_parsed_and_overwritten_part_1_of_2_run.csv",index=False)


In [ ]:
FDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_renamed_products_now_parsed_and_overwritten_part_1_of_2_run.csv")
FDF=FDF.rename(columns={'BASE_NAME':'NA','product':'BASE_NAME'})
SDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_renamed_products_NO_MAN_Product_names_parsed.csv")
SDF=SDF.drop_duplicates(subset="Product_Url",keep='first')
SDF=SDF[~SDF['BASE_NAME'].isna()]
SDF = SDF.dropna(subset=['Manufacturer','Product_Url','BASE_NAME'])
SDF=SDF.astype(str)
"""
Thought about "overweighting the name to reduce 1000 to 500 matches, but caused more noise 
"""
SDF['BASE_NAME']=SDF['BASE_NAME']+" :: "+SDF['Manufacturer']
FINAL=pd.merge(SDF,FDF,on='BASE_NAME',how='inner')

print(SDF.shape,FDF.shape,FINAL.shape)
FINAL.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_FINAL.csv",index=False)
DB=FINAL.loc[0:10000,:]
DB.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_FINAL_BUILD_DF.csv",index=False)

(88799, 57) (25675, 6) (88799, 62)


In [ ]:
FINAL.head()

In [ ]:
SDF.columns

Index(['Type', 'Name', 'Abbreviation', 'Capital', 'THREAD', 'Market_Focus',
       'DISPS_SRP', 'Pull_Date', 'Dispenary_base_leaf_url', 'Thread_List',
       'In_Market_Focus', 'Canada', 'Dispensary', 'Name_Tag', 'total_products',
       'Last_update', 'Member_Since', 'google_map', 'Total_Followers',
       'Address', 'Street', 'Town', 'State', 'Email', 'Phone', 'Website',
       'License', 'Leafly_review_count', 'FILE_NAME', 'total_pages',
       'products_on_last_page', 'Products_Page_URL', 'current_page', 'fop',
       'Page_Load_Tries', 'Product_Url', 'P_number', 'Variant', 'Product',
       'Product_size', 'Manufacturer', 'Category', 'Listed_content', 'THC%',
       'CBD%', 'UOM', 'Price', 'Collected_at', 'query_dirty', 'query',
       'Manufacturer_chk', 'TAG', 'BASE_NAME', 'Weight', 'Category_TAG',
       'Ratios', 'Pack_size'],
      dtype='object')

In [ ]:
# SDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_renamed_products_now_parsed.csv")
# print(SDF.columns)

# SDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_renamed_products_now_parsed.csv")
# SDF=SDF[['Dispensary','Product_size', 'Manufacturer', 'Category', 'Listed_content', 'THC%',
#       'CBD%', 'UOM', 'Price', 'query','Manufacturer_chk','Product_Url','mg',
#        'g', 'ratio', 'thc_cbd', 'pk_cont', 'pk', 'strain', 'oz']]
# print(SDF.shape)       
# print(SDF.columns)

# SDF=SDF.drop_duplicates(subset="Product_Url",keep='first')
# SDF=SDF[~SDF['query'].isna()]
# SDF = SDF.dropna(subset=['Manufacturer','Product_Url','query'])
# SDF=SDF.reset_index()
# SDF=SDF.drop(['index'],axis=1)

# print(SDF.shape)
  
# VC=pd.DataFrame(SDF['query'].value_counts())
# VC=VC.reset_index()
# #No drop to keep column index because it is the OEM's

# THREADS=CORES
# VC['THREAD']=VC.index%THREADS
# VC.head(100)
# MASTER_LIST=[]
# for i in range(THREADS):
#   temp=[]
#   tdf=VC[VC['THREAD']==i]
#   print(tdf.shape,VC.shape,len(MASTER_LIST))
#   tdf=tdf.copy()
#   tdf=tdf.reset_index()
#   #not index but level_0 since did not drop index last time
#   tdf=tdf.drop(['level_0'],axis=1)
#   tVC=VC.copy()
#   temp.append(tdf)
#   temp.append(tVC)
#   MASTER_LIST.append(temp)


# print(tdf.columns)

Index(['Dispensary', 'Name_Tag', 'total_products', 'Product_size',
       'Manufacturer', 'Category', 'Listed_content', 'THC%', 'CBD%', 'UOM',
       'Price', 'query', 'Manufacturer_chk', 'Product_Url', 'THREAD', 'mg',
       'g', 'ratio', 'thc_cbd', 'pk_cont', 'pk', 'strain', 'oz'],
      dtype='object')
(89035, 20)
Index(['Dispensary', 'Product_size', 'Manufacturer', 'Category',
       'Listed_content', 'THC%', 'CBD%', 'UOM', 'Price', 'query',
       'Manufacturer_chk', 'Product_Url', 'mg', 'g', 'ratio', 'thc_cbd',
       'pk_cont', 'pk', 'strain', 'oz'],
      dtype='object')
(89035, 20)
(741, 3) (29627, 3) 0
(741, 3) (29627, 3) 1
(741, 3) (29627, 3) 2
(741, 3) (29627, 3) 3
(741, 3) (29627, 3) 4
(741, 3) (29627, 3) 5
(741, 3) (29627, 3) 6
(741, 3) (29627, 3) 7
(741, 3) (29627, 3) 8
(741, 3) (29627, 3) 9
(741, 3) (29627, 3) 10
(741, 3) (29627, 3) 11
(741, 3) (29627, 3) 12
(741, 3) (29627, 3) 13
(741, 3) (29627, 3) 14
(741, 3) (29627, 3) 15
(741, 3) (29627, 3) 16
(741, 3) (29627, 3) 1

In [ ]:
# from numpy.core.fromnumeric import searchsorted
# SDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_renamed_products.csv")
# SDF['prod_mast_nm']="NOT_MATCHED_YET"
# SDF=SDF[['Dispensary', 'total_products','Product_Url', 'P_number', 'Variant', 'Product',
#        'Product_size', 'Manufacturer', 'Category', 'Listed_content', 'THC%',
#        'CBD%', 'UOM', 'Price', 'Collected_at', 'query_dirty', 'query',
#        'Manufacturer_chk', 'prod_mast_nm']]
# MC=pd.DataFrame(SDF['Manufacturer'].value_counts())
# MC.sort_values(['Manufacturer'],ascending=False,inplace=True)
# print(len(set(SDF['query'])))
# #### PREP
# ##   Sort by highest vc mans
# ##   spread evenly across threads
# ##   threaded function will rebuild main df
# MASTER_LIST=[]
# THREADS=CORES
# path="/content/drive/MyDrive/LEAFLY_Project/Data/THREAD_RESULTS/MAPPING"
# search="main_product_name_dispensary_side.csv"

# for m,man in enumerate(MC.index):
#   SDF.loc[SDF['Manufacturer']==man,"THREAD"]=int(m%THREADS)

# for i in range(THREADS):
#   temp=[]
#   tdf=SDF[SDF['THREAD']==i]
#   tdf=tdf.copy()
#   tdf['directory']=path
#   tdf['query_name']=search
#   tdf.reset_index(inplace=True)
#   tdf.drop(['index'],axis=1,inplace=True)
#   MASTER_LIST.append(tdf)


# def make_master_product_names(DF):
#   thread=DF.at[0,'THREAD']
#   MANS=list(set(DF['Manufacturer']))
#   #print(len(MANS))
#   for man in MANS:
#     tdf=DF[DF['Manufacturer']==man]
#     man_prod_list=list(set(tdf['query']))

    
#     for p,prod in enumerate(man_prod_list):
#       try:
#         if p>100:
#           break
#         try:
#           ops=process.extract(prod,man_prod_list)
#         except:
#           print(f"error {p} {product} ")
        
#         high_score=0
#         MATCHED=False
#         possibles=[]
#         for o,op in enumerate(ops):
    
#           if op[1]>high_score:
#             high_score=op[1]
#           if op[1]>=99:
#             MATCHED=True
#           if op[1]>=98 and op[0]!=prod:
#             possibles.append(op)
        
    
#         if len(possibles)>=1:
#           v1=prod
#           test=DF[DF['Manufacturer']==man]
#           pc=pd.DataFrame(test['query'].value_counts())
#           max=0
#           v1_count=pc.at[v1,'query']
#           for it, item in enumerate(possibles):
#             try:
#               print(f"{thread:4} {p:4} {prod[:25]:30} {prod==item[0]} {item[0][:25]:30} {item[1]}")
#             except:
#               print("print error")
#           #DF.loc[DF['query']==v1,'Final_Name']=v1
          
#         else:
#           DF.loc[DF['query']==prod,'Final_Name']=prod

#       except:
#         print("error in product loop",product)    
          

          

#       # if thread==0 and len(possibles)>1:
#       #   print(f"On MAN:{man:35} of {len(MANS):4} for product {prod:45} {p:4} of {len(man_prod_list):4} there are {len(possibles):4} possibles")

#   return DF

# def make_master_product_names_THREADED(MASTER_LIST):

#   threads=len(MASTER_LIST)

#   FDF=pd.DataFrame()
#   FMMDF=pd.DataFrame()
  
#   with concurrent.futures.ProcessPoolExecutor(max_workers=threads) as executor:
#     try:
#         for r,results in enumerate(executor.map(make_master_product_names,MASTER_LIST)):
#           FDF=FDF.append(results,ignore_index=True)
        
#         # path=FDF.at[0,'directory']
#         # file_name=FDF.at[0,'query_name']
#         # file_date_stamp=dt.datetime.now()
#         # file_date_stamp=file_date_stamp.strftime("%m_%d_%Y_%H_%M_")
#         # FDF.to_csv(join(path,f"_{file_name}"),index=False)
#         # FDF.to_csv(join(path,f"_{file_date_stamp}_{file_name}"),index=False)
#         print(FDF.info())
#         print(len(set(FDF['Final_Name'])))
#     except:
#         print("OOPS")

#   return FDF

# #   man_products=SDF.loc[SDF['Manufacturer']==man,'query'].to_list()
# #   for product in man_products:
# #     ops=process.extract(product,man_products)
# #     print(m,len(MANS),ops)
# FDF=make_master_product_names_THREADED(MASTER_LIST)

29628


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


 4.0    0 ll's bells popcorn   7g        False lls bells popcorn   7g         98
 2.0    4 milk chocolate jamocha 25      False milk chocolate jamocha 25      98
 4.0   11 lls bells popcorn   7g         False ll's bells popcorn   7g        98
 2.0    7 milk chocolate rolling 25      False milk chocolate rolling 25      98
12.0    4 bobby blue's buckshots 3.      False bobby blues buckshots 3.5      98
error 0 order online  
error in product loop
error 0 order online  
error in product loop
22.0    1 ev premium purple punch .      False ev premium purple punch ,      100
13.0   12 peanut butter 100mg            False peanut butter 100mg            100
10.0   26 swift lifts   mini pre ro      False swift lifts mini pre roll      98
12.0   26 frosty snowcones clear ca      False frosty snowcone clear car      98
13.0   13 brownie bites 400mg            False brownie bites 400mg            100
error 0 order online  
 6.0    4 's medicines tropical pun      False 's medicine tropical punc  

In [ ]:
# SDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_renamed_mans.csv")
# MAN_PROD=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/MASTER_PRODUCT_LSIT_renamed_mans.csv")

# SDF.loc[SDF['Manufacturer'].isna(),'Manufacturer']="TBD9999999999999999999999999999888888888888888888888888888888"
# SDF.loc[SDF['Manufacturer']=="-",'Manufacturer']="TBD99999999999999999999999999999999999999999999999999"
# #list of manufacturers
# disp_mans=list(set(SDF['Manufacturer']))
# mans=list(set(MAN_PROD['Manufacturer']))


# #DF lookup table for competing names
# VC=pd.DataFrame(SDF['Manufacturer'].value_counts())

# #depending on cpu use, Process pool or Thread pool will be used for asynchronous processing
# #keep in mind that a=b in one thread, b=a in another or in a different search.  resolve this
# # by picking the one with more products as the master name.  will require manual review of changes
# # but, can autmate the process with screen input once out of the step


# #Prepair data for asynchrounous processing
# THREADS=CORES

# query  = disp_mans.copy()
# search = mans.copy()

# QUERY  = pd.DataFrame(query ,columns=['Manufacturer'])
# SEARCH = pd.DataFrame(search,columns=['Manufacturer'])

# #configure 
# QUERY['THREAD']=QUERY.index%THREADS
# MASTER_LIST=[]
# for i in range(THREADS):
#   temp  = []
#   tdf   = QUERY[QUERY['THREAD']==i]
#   #send copies not pointers to original
#   tdf   = tdf.copy()
#   #easier to reference rows
#   tdf.reset_index(inplace=True)
#   tdf.drop(['index'],axis=1,inplace=True)
#   tdf['query_column'] = 'Manufacturer'
#   tdf['query_name']="product_manufacturer_vs_vendor_manufacturer_.csv"
#   tdf['directory'] = "/content/drive/MyDrive/LEAFLY_Project/Data/THREAD_RESULTS/MAPPING"
#   tdf['Multiple_Possible_Names']=False
#   tdf['MASTER_MAN_NAME']=tdf['Manufacturer']
#   tdf['algo_step']=1
#   SEARCH = SEARCH.copy()
#   SEARCH['search_column']='Manufacturer'
#   temp.append(tdf)
 
#   temp.append(SEARCH)
#   VC=VC.copy()
#   temp.append(VC)
#   MASTER_LIST.append(temp)


# FDF=process_for_match_THREADED(MASTER_LIST)

In [ ]:
# def pull_uoms(product,p,tdf,products="False"):

#   """
#   refactor these using \s{0,2} to reduce the number of searches
#   order them to overwrite a = b, b!=a
#   update tdf as found to ensure explicit recording

#   buil a coonversaion formula / function to convert g to mg for consistent labeling
#   """
  
#   units= {'ratio':  {'ratio':r'([^:]\d{1,4}\s{0,2}:\s{0,2}\d{1,4}[^:])','_ratio':r'(\d{1,4}\s:\s\d{1,4})',
#                     'ratioo':r'(\d{1,4}:\d{1,4}:\d{1,4})','_ratioo':r'(\d{1,4}\s:\s\d{1,4}:\d{1,4})',
#                     'ratiooo':r'(\d{1,4}:\d{1,4}:\d{1,4}:\d{1,4})','_ratiooo':r'(\d{1,4}\s:\s\d{1,4}:\d{1,4}:\d{1,4})'},
          
#           'thc_cbd':{'thc': r'(\d{1,3}mgthc/\d{1,3}mgcbd)',      'thc_': r'(\s\d{1,5}mgthc/\d{1,5}mg\scbd)',
#                    '_thc': r'(\s\d{1,5}mg\sthc/\d{1,5}mgcbd)','_thc_': r'(\s\d{1,5}mg\sthc/\d{1,5}mg\scbd)',
#                    '_thcwt':r'(\d{1,4}mgthc\s{0,1}:\s{0,1}\d{1,4}mgcbg)'},
          
#           'mg':     {'mg' :r'(\d{1,4}mg)','_mg':r'(\d{1,4}\smg)'},
          
#           'g':      {'g'  :r'(^\d\s\d{1,4}g)' ,'_g' :r'(^\d\s\d{1,4}\sg)',
#                     'decg'  :r'(^\d\s.\d{1}g)' ,'_decg' :r'(^\d\s.\d{1,4}\sg)',
#                     'gdecg'  :r'(\d{1}.\d{1,4}g)' ,'_gdecg' :r'(\d{1,4}.\d{1,4}\sg)'},
          
#           'oz':     {'oz' :r'(\d{1,4}oz)','_oz':r'(\d{1,4}\soz)',
#                     'frc_':r'(\d{1,4}/\d{1,4}\soz)', 'frcopslash':r'(\d{1,4}\\\d{1,4})',
#                     'frc' :r'(\d{1,3}/\d{1,3}oz)'},
          
#           'pk':     {'pk' :r'(\d{1,4}pk)','_pk':r'(\d{1,4}\spk)',
#                     'pack' :r'(\d{1,4}pack)','_pack':r'(\d{1,4}\spack)',
#                     'ct':r'(\d{1-4}ct)','_ct':r'(\d{1-4}\spack)'},
#           'pk_cont':{'count_x_measureg':'(\d{1,5}\sx\s\d{1,5}g)',
#                      'count_x_measuremg':'(\d{1,5}\sx\s\d{1,5}mg)'},
          
          
        
          
#           'strain' :{'i':r'(\si\s)','ih':'\si/h\s','sh':r'(\ss/h\s)','hs':r'(\sh/s\s)','hi':r'(\sh/i\s)','h':r'(\sh\s)'},
#   }
  
#   result=False
#   for k,v in units.items():
#     res="0"
#     for label,regex in v.items():
#       #print(f"look for this unit of measure {label} with this regex {regex}")
#       result = re.findall(regex, product)
  
#       if result:
#         for res in result:
#           tdf.at[0,k]=res.replace(k,'')
#           if p%500==0:
#             print(f"Product: {p:5} Unit: {k:6} {label:6} {res.replace(k,''):10} {product:50}")

#   return(tdf)


# def parse_products(LIST_o):

#   DFO=LIST_o[0]
#   #print(DFO.shape)
#   RDF=pd.DataFrame()

#   products=list(DFO['Product_Url'])


#   for p,product in enumerate(products):
    
#     tdf=DFO[DFO['Product_Url']==product]
#     tdf=tdf.copy()
#     tdf=tdf.reset_index()
#     tdf=tdf.drop(['index'],axis=1)
#     if len(tdf)<1:
#       print("ohhhhhhhhhhhhhhhhhh")
#     #print(tdf.shape)
#     #DFO=pull_uoms(product,p,SDF)
#     else:
#       query=tdf.at[0,'query']
#       pull_uoms(query,p,tdf)
#       RDF=RDF.append(tdf,ignore_index=True)

#   return(RDF)


# def MASTER_LIST_parse_products(THREADS):

#   SDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_renamed_products.csv")
#   SDF=SDF[['Dispensary', 'Name_Tag', 'total_products','Product_size', 'Manufacturer', 'Category', 'Listed_content', 'THC%',
#         'CBD%', 'UOM', 'Price', 'query','Manufacturer_chk','Product_Url']]
#   print(SDF.shape)       
#   print(SDF.columns)

#   SDF=SDF.drop_duplicates(subset="Product_Url",keep='first')
#   SDF=SDF[~SDF['query'].isna()]
#   SDF = SDF.dropna(subset=['query'])
#   SDF=SDF.reset_index()
#   SDF=SDF.drop(['index'],axis=1)

#   print(SDF.shape)
  
#   MASTER_LIST=[]
#   SDF['THREAD']=SDF.index%THREADS
#   for i in range(THREADS):
#     temp=[]
#     tdf=SDF[SDF['THREAD']==i]
#     print(tdf.shape)
#     tdf=tdf.copy()
#     tdf.reset_index(inplace=True)
#     tdf.drop(['index'],axis=1,inplace=True)
#     temp.append(tdf)
#     MASTER_LIST.append(temp)
    
#   return(MASTER_LIST)

# def parse_products_THREADED(MASTER_LIST):
#   threads=len(MASTER_LIST)
#   FDF=pd.DataFrame()
#   FMMDF=pd.DataFrame()
#   with concurrent.futures.ProcessPoolExecutor(max_workers=threads) as executor:
#     try:
#         for r,results in enumerate(executor.map(parse_products,MASTER_LIST)):
#           FDF=FDF.append(results,ignore_index=True)
#         print("TESTING")
#         # path=FDF.at[0,'directory']
#         # file_name=FDF.at[0,'query_name']
#         # file_date_stamp=dt.datetime.now()
#         # file_date_stamp=file_date_stamp.strftime("%m_%d_%Y_%H_%M_")
#         # FDF.to_csv(join(path,f"_{file_name}"),index=False)
#         # FDF.to_csv(join(path,f"_{file_date_stamp}_{file_name}"),index=False)
#         print(FDF.info())

#     except:
#         print("OOPS")
#   return(FDF)


# THREADS=CORES
# MASTER_LIST=MASTER_LIST_parse_products(THREADS)
# FDF=parse_products_THREADED(MASTER_LIST)

# FDF.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_renamed_products_now_parsed.csv",index=False)

In [ ]:
# FDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_renamed_products_now_parsed_and_overwritten_part_1_of_2_.csv")
# #FDF =pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_.csv")
# #regex=r"((((\.){0,1}\d{1,4}\.{0,1}\d{0,2}\s{0,1})(mg|g|oz|:|thc|cbd|cbn|pk))\s{0,1}\d{0,4}(mg|g\s|oz|:|thc|cbd|cbn))"
# #regex=r"((\.){0,1}\d{1,4}\.{0,1}\d{0,2})"
# """
# KEEP: 

# [ratio only regex]: r"(\d{1,5}\s{0,1}(:\s{0,1}\d{1,3}){1,5})"
# [ratio : / weight content (but picks up fractions too)] : r"((\d{1,5}\s{0,1}((mg|g\s|thc|cbd|cbg|cbn){0,2}\s{0,1}){1,2})((\/|:)\s{0,1}\d{1,5}\s{0,1}((mg|g\s|thc|cbd|cbg|cbn){0,2}\s{0,1}){1,2}){1,4})"
# [pack : g x count, or count]:r"(((\d{1,4}\s{0,1}(pk|pack))|(\s\d{0,3}\s{0,1}.{0,1}\d{0,2}(g\s|mg\s|oz\s))\s{0,1}x\s{0,1}\d{1,4})|((\s\d{0,3}\s{0,1}.{0,1}\d{0,2}x\s{0,1}\d{1,4}(g\s|mg\s|oz\s))\s{0,1}))"
# ==      FDF.at[i,"TAG"]+=f"+packsize:<{res[0].replace(' ','').replace('pack','pk')}>"
#         FDF.at[i,'BASE_NAME']=product.replace("[","").replace("]","")

# """
# FDF['BASE_NAME']=FDF['product']
# #Just ratio tags no content lables
# regex=r"(\s{0,1}\d{0,4}\s{0,1}\d{1,4}\s{0,1}\/\s{0,1}\d{1,4}\s{0,1}(g\s|oz\s|mg\s|\s))"
# FDF['TAG']=""
# for i in FDF.index:
#   product=FDF.at[i,'product']
#   # result = re.search(regex, product)
#   # if result:
#   #   print(result)
#   search=re.search(regex,product)
#   if search:
#     print(search[0],product)
#   # result = re.findall(regex, product)
#   # original = product
#   # if result:
#   #   for res in result:
#   #     #new=num2words(res[0])
#   #     product=product.replace(res[0],'',1)
#   #     FDF.at[i,"TAG"]+=f"+packsize:<{res[0].replace(' ','').replace('pack','pk')}>"
#   #     FDF.at[i,'BASE_NAME']=product.replace("[","").replace("]","")
#   #     # product=product+" "+new
#   #     print(f"{res[0]:20} >><<< {original:60}======{product:60}")

# #FDF.head(500)

1 1/4  1 1/4 classic rolling papers 
